# E2C Training

This is an interactive workflow for E2C training. Note that this specific case may generate results that are distinct from the paper.

During the training process (while the last cell is running), you can monitor the training status with Tensorboard. Make sure `tensorboard` is installed properly. To install `tensorboard`:  
`pip install tensorboard`  

All the data used for `tensorboard` are stored in `logs/` directory. If you do not have `logs/` directory in your cloned repo, please create one. To turn on `tensorboard`:  
`tensorboard --logdir=logs --port=5678` (`--port` is necesary for port-forwarding)




Zhaoyang Larry Jin  
Stanford University  
zjin@stanford.edu

### Step 1. Load libaraies and config hardware (gpu)

In [ ]:
import numpy as np
import h5py

import e2c as e2c_util

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1", "2", "3"

import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

# GPU memory management
import tensorflow as tf
from datetime import datetime


from train import *

In [ ]:
# tf.session specification
# TensorFlow wizardry
config = tf.ConfigProto()

# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.75

# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))

## Step 2. Specify params and load data

In [ ]:
################### case specification ######################

data_dir = '../data/'
output_dir = './saved_models/'

case_name = '9w_ms_bhp_rate'
case_suffix = '_fix_wl_rel_8'
train_suffix = '_with_p'
model_suffix = '_flux_loss'

n_train_run = 300
n_eval_run = 100
num_t = 20 
dt = 100
n_train_step = n_train_run * num_t
n_eval_step = n_eval_run * num_t


train_file = case_name + '_e2c_train' + case_suffix + train_suffix + '_n%d_dt%dday_nt%d_nrun%d.mat' %(n_train_step, dt, num_t, n_train_run)
eval_file = case_name + '_e2c_eval' + case_suffix + train_suffix +'_n%d_dt%dday_nt%d_nrun%d.mat' %(n_eval_step, dt, num_t, n_eval_run)

In [ ]:
#################### model specification ##################
epoch = 10
batch_size = 4
learning_rate = 1e-4
latent_dim = 50

u_dim = 9*2 # control dimension, gaussian 9 wells

# load data
hf_r = h5py.File(data_dir + train_file, 'r')
state_t_train = np.array(hf_r.get('state_t'))
state_t1_train = np.array(hf_r.get('state_t1'))
bhp_train = np.array(hf_r.get('bhp'))
dt_train = np.array(hf_r.get('dt'))
hf_r.close()

num_train = state_t_train.shape[0]
# dt_train = np.ones((num_train,1)) # dt=20days, normalized to 1

hf_r = h5py.File(data_dir + eval_file, 'r')
state_t_eval = np.array(hf_r.get('state_t'))
state_t1_eval = np.array(hf_r.get('state_t1'))
bhp_eval = np.array(hf_r.get('bhp'))
dt_eval = np.array(hf_r.get('dt'))
hf_r.close()

print("state_t_eval.shape: ", state_t_eval.shape)
print("state_t1_eval.shape: ", state_t1_eval.shape)
print("bhp_eval.shape: ", bhp_eval.shape)
print("dt_eval.shape: ", dt_eval.shape)


num_eval = state_t_eval.shape[0]
# dt_eval = np.ones((num_eval, 1)) # dt=20days, normalized to 1

#### Load permeability data

In [ ]:
m = np.loadtxt("../data/template/logk1.dat") # Gaussian

m = m.reshape(60, 60, 1)
print('m shape is ', m.shape)

m_tf = Input(shape=(60, 60, 1))

m_eval = np.repeat(np.expand_dims(m, axis = 0), state_t_eval.shape[0], axis = 0)
print("m_eval shape is ", m_eval.shape)

m = np.repeat(np.expand_dims(m,axis = 0), state_t_train.shape[0], axis = 0)
print("m shape is ", m.shape)

#### Load well location data

In [ ]:
well_loc_file = '../data/template/well_loc00.dat'

well_loc = np.loadtxt(well_loc_file).astype(int)
num_prod = well_loc[0,0]
num_inj = well_loc[0,1]
num_well = num_prod+num_inj
print(num_inj, num_prod)

prod_loc = well_loc[1:num_prod+1,:]
print("prod_loc:\n{}".format(prod_loc))
print(prod_loc.shape)

print('prod_loc shape is ', prod_loc.shape)
prod_loc_tf = tf.placeholder(tf.int32, shape=(num_prod,2))


## Step 3. Construct E2C model and loss function

In [ ]:

input_shape = (60, 60, 2)

#############################################
# Note
# For E2C with AE, sigma = 0.0
# For E2C with UAE framework, sigma != 0.0, (e.g., =0.001)
#############################################
encoder, decoder, transition = create_e2c(latent_dim, u_dim, input_shape, sigma=0.0) 


xt = Input(shape=input_shape)
xt1 = Input(shape=input_shape)
ut = Input(shape=(u_dim, ))
dt = Input(shape=(1,))

zt = encoder(xt)
xt_rec = decoder(zt)

zt1 = encoder(xt1)

zt1_pred = transition([zt, ut, dt])
xt1_pred = decoder(zt1_pred)

# Compute loss
loss_rec_t = reconstruction_loss(xt, xt_rec)
loss_rec_t1 = reconstruction_loss(xt1, xt1_pred)

loss_flux_t = get_flux_loss(m_tf, xt, xt_rec) / 1000.
loss_flux_t1 = get_flux_loss(m_tf, xt1, xt1_pred) / 1000.

binary_sat_loss_t = get_binary_sat_loss(xt, xt_rec) * 1
binary_sat_loss_t1 = get_binary_sat_loss(xt1, xt1_pred) * 1

loss_prod_bhp_t = get_well_bhp_loss(xt, xt_rec, prod_loc_tf) * 20
loss_prod_bhp_t1 = get_well_bhp_loss(xt1, xt1_pred, prod_loc_tf) * 20

loss_l2_reg = l2_reg_loss(zt)  # log(1.) = 0.

loss_bound = loss_rec_t + loss_rec_t1 + loss_l2_reg  + loss_flux_t + loss_flux_t1 + loss_prod_bhp_t + loss_prod_bhp_t1 # JPSE 2020 Gaussian case

#####################################################################
# Note: you can also use other combination to construct loss function
# loss_bound = loss_rec_t + loss_rec_t1  + loss_flux_t + loss_flux_t1 + loss_prod_bhp_t + loss_prod_bhp_t1 # UAE
# loss_bound = loss_rec_t + loss_rec_t1 + loss_l2_reg # no flux/bhp loss comparison
# loss_bound = loss_rec_t + loss_rec_t1 + loss_l2_reg  + loss_flux_t + loss_flux_t1
# loss_bound = loss_rec_t + loss_rec_t1 + loss_kl + binary_sat_loss_t + binary_sat_loss_t1
#####################################################################

# Use zt_logvar to approximate zt1_logvar_pred
loss_trans = l2_reg_loss(zt1_pred - zt1)
# loss_trans = kl_normal_loss(zt1_mean_pred, zt1_logvar_pred, zt1_mean, zt1_logvar)


trans_loss_weight = 1.0 # lambda in E2C paper Eq. (11)
loss = loss_bound + trans_loss_weight * loss_trans

#### Create log for `Tensorboard`

In [ ]:

def write_summary(value, tag, summary_writer, global_step):
    """Write a single summary value to tensorboard"""
    summary = tf.Summary()
    summary.value.add(tag=tag, simple_value=value)
    summary_writer.add_summary(summary, global_step)

## used to generate log directory
currentDT = datetime.now()
current_time = str(currentDT).replace(" ", "-")[:-10]
print(current_time)

summary_writer = tf.summary.FileWriter('logs/' + case_name + case_suffix + '_ep' + str(epoch) + '_tr' + str(n_train_run) + '_' + current_time)

#### Construct computation graph (only necessary for `Tensorflow 1.x`)

In [ ]:
# Optimization
opt = Adam(lr=learning_rate)

trainable_weights = encoder.trainable_weights + decoder.trainable_weights + transition.trainable_weights

updates = opt.get_updates(loss, trainable_weights)

iterate = K.function([xt, ut, xt1, m_tf, prod_loc_tf, dt], [loss, loss_rec_t, loss_rec_t1, loss_l2_reg, loss_trans, loss_flux_t, loss_flux_t1, loss_prod_bhp_t, loss_prod_bhp_t1], updates=updates)

eval_loss = K.function([xt, ut, xt1, m_tf, prod_loc_tf, dt], [loss])

num_batch = int(num_train/batch_size)



## Step 4. Begin training

In [ ]:
for e in range(epoch):
    for ib in range(num_batch):
        ind0 = ib * batch_size
        state_t_batch  = state_t_train[ind0:ind0+batch_size, ...]
        state_t1_batch = state_t1_train[ind0:ind0 + batch_size, ...]
        bhp_batch      = bhp_train[ind0:ind0 + batch_size, ...]
        m_batch        = m[ind0:ind0 + batch_size, ...]
        dt_batch       = dt_train[ind0:ind0 + batch_size, ...]
        
        output = iterate([state_t_batch, bhp_batch, state_t1_batch, m_batch, prod_loc, dt_batch])
        
        n_itr = e * num_train + ib * batch_size + batch_size
        write_summary(output[0], 'train/total_loss', summary_writer, n_itr) # log for tensorboard
        write_summary(output[1]+output[2], 'train/sum_rec_loss', summary_writer, n_itr) # log for tensorboard
        write_summary(output[5]+output[6], 'train/sum_flux_loss', summary_writer, n_itr) # log for tensorboard
        write_summary(output[7]+output[8], 'train/sum_well_loss', summary_writer, n_itr) # log for tensorboard

        if ib % 50 == 0:
            print('Epoch %d/%d, Batch %d/%d, Loss %f, Loss rec %f, loss rec t1 %f, loss kl %f, loss_trans %f, loss flux %f, loss flux t1 %f, prod bhp loss %f, prod bhp loss t1 %f' % (e+1, epoch, ib+1, num_batch, output[0], output[1], output[2], output[3], output[4], output[5], output[6], output[7], output[8]))
            eval_loss_val = eval_loss([state_t_eval, bhp_eval, state_t1_eval, m_eval, prod_loc, dt_eval])
            write_summary(eval_loss_val[0], 'eval/total_loss', summary_writer, n_itr) # log for tensorboard
    
    print('====================================================')
    print('\n')
    print('Epoch %d/%d, Train loss %f, Eval loss %f' % (e + 1, epoch, output[0], eval_loss_val[0]))
    print('\n')
    print('====================================================')

encoder.save_weights(output_dir + 'e2c_encoder_dt_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
decoder.save_weights(output_dir + 'e2c_decoder_dt_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
transition.save_weights(output_dir + 'e2c_transition_dt_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                        % (num_train, latent_dim, learning_rate, epoch))